In [14]:
import traceback
from time import sleep
import time
import threading
import serial
import numpy as np
from datetime import datetime
from feagi_connector import feagi_interface as feagi
from feagi_connector import sensors
from feagi_connector import pns_gateway as pns
from feagi_connector.version import __version__
from feagi_connector import actuators
import ok
from OpenCat.serialMaster.ardSerial import *

In [ ]:
dev = ok.okCFrontPanel()
dev.OpenBySerial("")
error = dev.ConfigureFPGA(r"C:\Users\ereij\OneDrive\Documents\PITT\FPGA\qDVS\CPG_OK.bit")
print(error)

In [ ]:
# set timing
count = 0x08000000
clk_f = 200E6
tc = int(count)/clk_f
print(int(count), tc)

def bitfield(n, length = 16):
    bf = [int(digit) for digit in bin(n)[2:]]
    for i in range(length - len(bf)):
        bf = [0] + bf
    return bf


# Reset chip
dev.SetWireInValue(0x01, count)    # sets count
dev.SetWireInValue(0x00, 0b0010)        # pull both resets and spike in down
dev.UpdateWireIns()

time.sleep(tc)
dev.SetWireInValue(0x00, 0b1000)        # pull clk_rst_n high

time.sleep(tc*10)
dev.SetWireInValue(0x00, 0b1101)        # pull rst_n high and start high
dev.UpdateWireIns()

time.sleep(tc*1.6)
dev.SetWireInValue(0x00, 0b1100)        # pull start low
dev.UpdateWireIns()



In [ ]:
old_data = 0
t_0 = time.time()
steps = []

while len(steps) < 16:
    data = dev.UpdateWireOuts()
    data = dev.GetWireOutValue(0x20)
    if data != old_data:
        print(len(steps), bitfield(data), len(bitfield(data)), time.time()-t_0)
        t_0 = time.time()
        steps.append(bitfield(data))
    old_data = data

In [ ]:
# for point in steps:
#     print(point)
steps

In [ ]:
from serial.tools import list_ports
port = list(list_ports.comports())
for p in port:
    print(p.device)

In [1]:
import traceback
from time import sleep
import time
import threading
import serial
import numpy as np
from datetime import datetime
from feagi_connector import feagi_interface as feagi
from feagi_connector import sensors
from feagi_connector import pns_gateway as pns
from feagi_connector.version import __version__
from feagi_connector import actuators
import ok
# i 0 4 1 -1 2 3 3 0 4 0 5 0 6 0 7 0 8 29 9 31 10 31 11 29 12 25 13 23 14 26 15 25
servo_status = [30,	30,	30,	30,	30,	30,	30,	30]
gyro = {}

# set timing
count = 0x0800000
clk_f = 200E6
tc = int(count) / clk_f
# print(int(count), tc)

# Program FPGA
try:
    print("Programming FPGA...")
    print("\tTime Constant: ", str(tc))
    dev = ok.okCFrontPanel()
    dev.OpenBySerial("")
    error = dev.ConfigureFPGA(r"C:\Users\ereij\OneDrive\Documents\PITT\FPGA\qDVS\CPG_OK.bit")

    print("Connecting to Bot...")
    ser = serial.Serial('COM5', 115200)
    time.sleep(5)
    ser.write('i 0 4 1 -1 2 3 3 0 4 0 5 0 6 0 7 0 8 29 9 31 10 31 11 29 12 25 13 23 14 26 15 25'.encode())

    print("Starting now!")
except Exception as Error_case:
    print(Error_case)
    ser.write('kbalance'.encode())
    dev.Close()
    ser.close()
    exit()

def simulation_from_fpga():
    return [np.random.choice([0, 1]) for _ in range(16)]


def bitfield(n, length = 16):
    bf = [int(digit) for digit in bin(n)[2:]]
    for i in range(length - len(bf)):
        bf = [0] + bf
    return bf
# Function to handle receiving data
def read_from_port(ser='', dev = ''):
    global received_data, gyro
    # received_data = 0
    full_data = ''

    try:
        # Reset chip
        dev.SetWireInValue(0x01, count)  # sets count
        dev.SetWireInValue(0x00, 0b0010)  # pull both resets and spike in down
        dev.UpdateWireIns()

        time.sleep(tc)
        dev.SetWireInValue(0x00, 0b1000)  # pull clk_rst_n high

        time.sleep(tc * 10)
        dev.SetWireInValue(0x00, 0b1101)  # pull rst_n high and start high
        dev.UpdateWireIns()

        time.sleep(tc * 1.6)
        dev.SetWireInValue(0x00, 0b1100)  # pull start low
        dev.UpdateWireIns()

        old_data = 0
        t_0 = time.time()
        steps = []
        while True:
            dev.UpdateWireOuts()
            data = dev.GetWireOutValue(0x20)
            if data != old_data:
                # print(bitfield(data), len(bitfield(data)), time.time() - t_0)
                t_0 = time.time()
                # steps.append(bitfield(data))
                reading = bitfield(data)
                received_data = reading
                full_data = received_data
                action(full_data)
            old_data = data
    except Exception as Error_case:
        pass
        print(Error_case)
        dev.Close()
        exit()
        # counter += 1
    except KeyboardInterrupt:
        dev.Close()
        ser.write('kbalance'.encode())
        print("keyboard interrupt".capitalize())
        exit()

def feagi_to_petoi_id(device_id):
    mapping = {
        0: 8,
        1: 12,
        2: 9,
        3: 13,
        4: 11,
        5: 15,
        6: 10,
        7: 14
    }
    return mapping.get(device_id, None)


def action(obtained_data):
    try:
        # fpga here section:
        servo_for_feagi = 'i '
        print("full raw data: ", obtained_data)
        for servo_id in range(0, len(obtained_data), 2):
            mapped_id = feagi_to_petoi_id(servo_id // 2)
            value1, value2 = obtained_data[servo_id], obtained_data[servo_id + 1]
            turn = 0
            if mapped_id in [8, 9, 10, 11]:
                turn = 20
            if mapped_id in [12, 13, 14, 15]:
                turn = 45
            if value1 == 1:
                print(servo_id, value1)
                servo_status[servo_id // 2] -= turn
            if value2 == 1:
                print(servo_id+1, value2)
                servo_status[servo_id // 2] += turn
            servo_status[servo_id // 2] = actuators.servo_keep_boundaries(servo_status[servo_id // 2], 90, -90) # block from exceeded 90
            # Append the mapped ID and the adjusted status to the result string
            servo_for_feagi += str(mapped_id) + " " + str(servo_status[servo_id // 2]) + " "
        # print("final: ", servo_for_feagi.encode())
        print("final: ", servo_status)
        ser.write(servo_for_feagi.encode())

    except KeyboardInterrupt:
        ser.write('kbalance'.encode())
        ser.close()
        print("keyboard interrupt".capitalize())
        exit()


if __name__ == "__main__":
    # To give ardiuno some time to open port. It's required
    try:
        # for x in range(8):
        #     servo_status.append(0)
        read_from_port(dev=dev)
    except Exception as Error_case:
        print(Error_case, "FPGA: ",error)
        ser.write('kbalance'.encode())
        ser.close()
        exit()
    except KeyboardInterrupt:
        ser.write('kbalance'.encode())
        ser.close()
        print("keyboard interrupt".capitalize())
        exit()


Programming FPGA...
	Time Constant:  0.04194304
Connecting to Bot...
Starting now!
full raw data:  [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
4 1
8 1
final:  [30.0, 30.0, 10.0, 30.0, 10.0, 30.0, 30.0, 30.0]
full raw data:  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
6 1
10 1
final:  [30.0, 30.0, 10.0, -15.0, 10.0, -15.0, 30.0, 30.0]
full raw data:  [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
0 1
5 1
9 1
12 1
final:  [10.0, 30.0, 30.0, -15.0, 30.0, -15.0, 10.0, 30.0]
full raw data:  [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
2 1
7 1
11 1
14 1
final:  [10.0, -15.0, 30.0, 30.0, 30.0, 30.0, 10.0, -15.0]
full raw data:  [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
1 1
4 1
8 1
13 1
final:  [30.0, -15.0, 10.0, 30.0, 10.0, 30.0, 30.0, -15.0]
full raw data:  [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
3 1
6 1
10 1
15 1
final:  [30.0, 30.0, 10.0, -15.0, 10.0, -15.0, 30.0, 30.0]
full raw data:  [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
0 1
5 1
9 1
12 1
final:

In [1]:
# dev.Close()
ser.close()

NameError: name 'ser' is not defined

In [18]:
wkF = [
        54, 0, 0, 1,# 54 is the number of frames. it should not be larger than 127
        9,  49,  53,  45,  24,  20,  -2,  15,
        8,  50,  41,  46,  28,  21,  -1,  15,
        10,  51,  26,  47,  26,  22,   6,  16,
        12,  52,  23,  48,  24,  24,   9,  17,
        14,  52,  20,  49,  22,  26,  12,  18,
        16,  53,  17,  51,  21,  27,  17,  18,
        18,  53,  14,  52,  20,  29,  22,  19,
        21,  54,  11,  54,  18,  30,  27,  19,
        22,  54,  11,  54,  18,  32,  29,  20,
        25,  54,  13,  55,  16,  34,  27,  21,
        26,  54,  16,  56,  16,  37,  24,  23,
        28,  54,  18,  56,  15,  39,  23,  24,
        30,  52,  20,  57,  14,  45,  22,  26,
        32,  54,  22,  57,  14,  44,  21,  28,
        33,  58,  24,  57,  15,  36,  20,  30,
        34,  61,  26,  57,  15,  31,  19,  32,
        36,  64,  28,  57,  14,  24,  18,  35,
        38,  66,  29,  57,  14,  20,  18,  38,
        39,  67,  31,  57,  14,  16,  17,  40,
        41,  64,  32,  56,  14,   5,  17,  43,
        42,  55,  35,  57,  14,  -1,  16,  44,
        44,  44,  37,  62,  15,  -3,  14,  35,
        45,  30,  39,  66,  15,   1,  14,  29,
        46,  21,  40,  68,  15,   5,  14,  23,
        47,  19,  42,  70,  16,   9,  14,  19,
        48,  16,  43,  70,  17,  12,  15,  17,
        49,  12,  44,  67,  18,  17,  15,   5,
        49,   9,  46,  59,  20,  24,  15,  -2,
        50,   8,  47,  47,  21,  28,  16,  -2,
        51,  10,  48,  34,  22,  26,  16,   1,
        52,  12,  49,  24,  24,  24,  17,   6,
        52,  14,  50,  21,  26,  22,  18,  10,
        53,  16,  51,  19,  27,  21,  19,  12,
        53,  18,  52,  15,  29,  20,  20,  19,
        54,  21,  54,  12,  30,  18,  19,  24,
        54,  22,  55,  12,  32,  18,  20,  27,
        54,  25,  55,  11,  34,  16,  22,  29,
        54,  26,  56,  14,  37,  16,  24,  26,
        54,  28,  56,  17,  39,  15,  25,  24,
        52,  30,  57,  18,  45,  14,  27,  23,
        54,  32,  57,  21,  44,  14,  29,  21,
        58,  33,  57,  23,  36,  15,  31,  20,
        61,  34,  57,  24,  31,  15,  33,  20,
        64,  36,  57,  26,  24,  14,  36,  19,
        66,  38,  57,  28,  20,  14,  39,  18,
        67,  39,  56,  30,  16,  14,  42,  17,
        64,  41,  56,  32,   5,  14,  45,  17,
        55,  42,  59,  33,  -1,  14,  41,  17,
        44,  44,  64,  35,  -3,  15,  33,  16,
        30,  45,  67,  38,   1,  15,  27,  14,
        21,  46,  68,  39,   5,  15,  22,  14,
        19,  47,  70,  41,   9,  16,  18,  14,
        16,  48,  69,  42,  12,  17,  11,  14,
        12,  49,  63,  44,  17,  18,   1,  15]
goodPorts = {}
connectPort(goodPorts)
send(goodPorts, ['k', wkF])
print(goodPorts)
send(goodPorts, ['kbalance',0])
closeAllSerial(goodPorts)

Exception in thread Thread-23 (testPort):
Traceback (most recent call last):
  File "C:\Users\ereij\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\ereij\OneDrive\Documents\PITT\FPGA\venv\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\ereij\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ereij\OneDrive\Documents\PITT\FPGA\OpenCat\serialMaster\ardSerial.py", line 606, in testPort
    raise e
  File "C:\Users\ereij\OneDrive\Documents\PITT\FPGA\OpenCat\serialMaster\ardSerial.py", line 599, in testPort
    getModelAndVersion(result)
  File "C:\Users\ereij\OneDrive\Documents\PITT\FPGA\OpenCat\serialMaster\ardSerial.py", line 554, in getModelAndVersion
    config.model_ = parse[l]
    ^^^^^^
NameError: name 'config' is not defined


* Port COM5 cannot be opened!
{<OpenCat.serialMaster.SerialCommunication.Communication object at 0x000001BC2F07CA50>: 'COM5'}


In [8]:
closeAllSerial(goodPorts)

In [1]:
# forever test -- ideal data

import traceback
from time import sleep
import time
import threading
import serial
import numpy as np
from datetime import datetime
from feagi_connector import feagi_interface as feagi
from feagi_connector import sensors
from feagi_connector import pns_gateway as pns
from feagi_connector.version import __version__
from feagi_connector import actuators
import ok

steps = [[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
         [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
         [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
         [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
         [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]]

def feagi_to_petoi_id(device_id):
    mapping = {
        0: 8,
        1: 12,
        2: 9,
        3: 13,
        4: 11,
        5: 15,
        6: 10,
        7: 14
    }
    return mapping.get(device_id, None)

def action(obtained_data):
    try:
        # fpga here section:
        servo_for_feagi = 'i '
        print("full raw data: ", obtained_data)
        for servo_id in range(0, len(obtained_data), 2):
            mapped_id = feagi_to_petoi_id(servo_id // 2)
            value1, value2 = obtained_data[servo_id], obtained_data[servo_id + 1]
            turn = 0
            if mapped_id in [8, 9, 10, 11]:
                turn = 15
            if mapped_id in [12, 13, 14, 15]:
                turn = 30
            if value1 == 1:
                # print(servo_id, value1)
                servo_status[servo_id // 2] -= turn
            if value2 == 1:
                # print(servo_id+1, value2)
                servo_status[servo_id // 2] += turn
            servo_status[servo_id // 2] = actuators.servo_keep_boundaries(servo_status[servo_id // 2], 90, -90) # block from exceeded 90
            # Append the mapped ID and the adjusted status to the result string
            servo_for_feagi += str(mapped_id) + " " + str(servo_status[servo_id // 2]) + " "
        # print("final: ", servo_for_feagi.encode())
        print("final: ", servo_status)
        ser.write(servo_for_feagi.encode())

    except KeyboardInterrupt:
        ser.write('kbalance'.encode())
        ser.close()
        print("keyboard interrupt".capitalize())
        exit()

servo_status = [30,	30,	30,	30,	30,	30,	30,	30]
gyro = {}
count = 0x200000
clk_f = 200E6
tc = int(count) / clk_f
print("Time Constant: ", tc)

try:
    print("Connecting to Bot...")
    ser = serial.Serial('COM5', 115200)
    time.sleep(5)
    ser.write('i 0 4 1 -1 2 3 3 0 4 0 5 0 6 0 7 0 8 29 9 31 10 31 11 29 12 25 13 23 14 26 15 25'.encode())
    idx = 0
    while True:
        action(steps[idx])
        idx += 1
        if idx > 15:
            idx = 0
        time.sleep(tc)
        # print(ser.reset_input_buffer())
except Exception as error:
    print(error)
    ser.close()
    exit()

Time Constant:  0.01048576
Connecting to Bot...
full raw data:  [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
3 1
6 1
10 1
15 1
final:  [30.0, 60.0, 30.0, 0.0, 30.0, 0.0, 30.0, 60.0]
full raw data:  [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
0 1
5 1
9 1
12 1
final:  [15.0, 60.0, 45.0, 0.0, 45.0, 0.0, 15.0, 60.0]
full raw data:  [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
2 1
7 1
11 1
14 1
final:  [15.0, 30.0, 45.0, 30.0, 45.0, 30.0, 15.0, 30.0]
full raw data:  [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
1 1
4 1
8 1
13 1
final:  [30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0]
full raw data:  [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
3 1
6 1
10 1
15 1
final:  [30.0, 60.0, 30.0, 0.0, 30.0, 0.0, 30.0, 60.0]
full raw data:  [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
0 1
5 1
9 1
12 1
final:  [15.0, 60.0, 45.0, 0.0, 45.0, 0.0, 15.0, 60.0]
full raw data:  [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
2 1
7 1
11 1
14 1
final:  [15.0, 30.0, 45.0, 30.0, 45.0, 3

KeyboardInterrupt: 

In [17]:
count = 0x200000
clk_f = 200E6
tc = int(count) / clk_f
tc

0.01048576

In [18]:
print(0x200000/0x2000000)

0.0625
